<a href="https://colab.research.google.com/github/adamgrzanek/predicting_car_data/blob/main/cars_ads_otomoto_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading car ads from otomoto.pl

The main goal is to retrieve a data frame with information about cars

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from time import sleep
from matplotlib import pyplot as plt

## Checking the page layout

In [3]:
page_number = 1
url = f"https://www.otomoto.pl/osobowe/uzywane/seg-mini/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"

In [4]:
response = requests.get(url=url)
response

<Response [200]>

In [5]:
content = response.text
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
print(soup.prettify())

In [7]:
offers = soup.find_all('h1', {'class': "ev7e6t89 ooa-1xvnx1e er34gjf0"})

In [8]:
len(offers)

36

In [11]:
print(offers[0].prettify())

<h1 class="ev7e6t89 ooa-1xvnx1e er34gjf0">
 <a href="https://www.otomoto.pl/osobowe/oferta/citroen-c3-1-4-hdi-68km-lift-led-klimatronic-navi-tempomat-ID6FPXr1.html" rel="noreferrer" target="_self">
  Citroën C3 1.4 HDi Attraction
 </a>
</h1>



## Checking the offer page layout

In [10]:
# example offer url
offer_url = offers[0].find('a')['href']
offer_url

'https://www.otomoto.pl/osobowe/oferta/citroen-c3-1-4-hdi-68km-lift-led-klimatronic-navi-tempomat-ID6FPXr1.html'

In [12]:
offer_response = requests.get(url=offer_url)
offer_response

<Response [200]>

In [ ]:
offer_content = offer_response.text
offer_soup = BeautifulSoup(offer_content, 'html.parser')

print(offer_soup.prettify())

## Scraping data from the offer
('script', {'id': "__NEXT_DATA__"})

In [14]:
text = offer_soup.find('script', {'id': "__NEXT_DATA__"}).get_text()

In [ ]:
data_json = json.loads(text)
data_json

In [ ]:
data_json['props']['pageProps']['advert']

### Price

In [17]:
data_json['props']['pageProps']['advert']['price']

{'value': '12999',
 'currency': 'PLN',
 'labels': [],
 'type': 'gross',
 'isUnderBudget': False}

In [18]:
price = float(data_json['props']['pageProps']['advert']['price']['value'])
price

12999.0

### Basic parameters

In [ ]:
offer_params = data_json['props']['pageProps']['advert']['details']
offer_params

In [20]:
offer_params[0]

{'key': 'private_business',
 'label': 'Oferta od',
 'value': 'Prywatne',
 'href': 'https://www.otomoto.pl/osobowe/?search%5Bprivate_business%5D=private'}

In [21]:
offer_data = {}
for i in offer_params:
    label = i['label']
    value = i['value']
    offer_data[label] = value

offer_data

{'Oferta od': 'Prywatne',
 'Importowany': 'Tak',
 'Pokaż oferty z numerem VIN': 'Tak',
 'Ma numer rejestracyjny': 'Tak',
 'Marka pojazdu': 'Citroën',
 'Model pojazdu': 'C3',
 'Wersja': '1.4 HDi Attraction',
 'Generacja': 'II (2008-2018)',
 'Rok produkcji': '2014',
 'Przebieg': '167 500 km',
 'Pojemność skokowa': '1 398 cm3',
 'VIN': 'VF7SC8HR4EW648604',
 'Rodzaj paliwa': 'Diesel',
 'Moc': '68 KM',
 'Skrzynia biegów': 'Manualna',
 'Napęd': 'Na przednie koła',
 'Spalanie Poza Miastem': '3.0 l/100km',
 'Spalanie W Mieście': '4 l/100km',
 'Typ nadwozia': 'Auta małe',
 'Emisja CO2': '99 g/km',
 'Liczba drzwi': '5',
 'Liczba miejsc': '2',
 'Kolor': 'Srebrny',
 'Rodzaj koloru': 'Metalik',
 'Kraj pochodzenia': 'Francja',
 'Data pierwszej rejestracji w historii pojazdu': '24 wrzesień 2014',
 'Zarejestrowany w Polsce': 'Tak',
 'Stan': 'Używane'}

### Description

In [22]:
description = data_json['props']['pageProps']['advert']['description'].replace('\n', ' ')
description

'!!! WERSJA 2 OSOBOWA - BRAK TYLNEJ KANAPY !!!  CITROEN C3 II HATCHBACK SOCIETE PO FACELIFTINGU 1.4 HDi 68 KM (50kW) DIESEL  Pierwsza rejestracja: 24.09.2014 Rok produkcji: 2014  Przegląd do 12.2023 OC do 07.2024  Nr rej.: CB322JW  VIN - podany wyżej w szczegółach pojazdu !!!  Kolor - SREBRNY METALIK  Przebieg - 167 500 km (oryginalny przebieg)  Sprowadzony z Francji w 2019  Wyposażenie:  - SOCIETE - 2 osobowy - 5-cio biegowa skrzynia biegów - światła LED do jazdy dziennej - ABS - ESP - regulowana kierownica - 4 x AIRBAG - 2 x el.szyby - el.lusterka (podgrzewane) - klimatronic (automatyczna klimatyzacja) - klimatyzowany schowek - komputer pokładowy - regulacja wysokości fotela kierowcy - immobilizer - TEMPOMAT - oryginalne radio CD + Mp3 + USB - oryginalna KOLOROWA NAWIGACJA - zestaw głośnomówiący na Bluetooth - centralny zamek na pilota - 2 szt oryginalnych kluczy  Cena: 12 999 zł  Zapraszam na oględziny i jazdę próbną !!!'

### Seller

In [23]:
seller = data_json['props']['pageProps']['advert']['seller']
seller

{'type': 'PRIVATE',
 'name': 'Radosław',
 'featuresBadges': [{'code': 'private-seller', 'label': 'Osoba prywatna'},
  {'code': 'fast-reply', 'label': 'Bardzo sprawnie odpowiada'},
  {'code': 'registration-date', 'label': 'Sprzedający na OTOMOTO od 2015'}],
 'logos': [],
 'location': {'address': 'Bydgoszcz, Kujawsko-pomorskie',
  'city': 'Bydgoszcz',
  'cityId': 4019,
  'region': 'Kujawsko-pomorskie',
  'regionId': 15,
  'country': None,
  'postalCode': None,
  'shortAddress': 'Bydgoszcz, Kujawsko-pomorskie',
  'map': {'latitude': 53.1342,
   'longitude': 17.99035,
   'zoom': 13,
   'radius': 1500}},
 'id': '1740079',
 'uuid': 'b389755d-56a7-4ddc-9d25-be2da4f52eb0'}

In [24]:
seller['location']

{'address': 'Bydgoszcz, Kujawsko-pomorskie',
 'city': 'Bydgoszcz',
 'cityId': 4019,
 'region': 'Kujawsko-pomorskie',
 'regionId': 15,
 'country': None,
 'postalCode': None,
 'shortAddress': 'Bydgoszcz, Kujawsko-pomorskie',
 'map': {'latitude': 53.1342,
  'longitude': 17.99035,
  'zoom': 13,
  'radius': 1500}}

In [25]:
seller_province = seller['location']['region']
seller_province

'Kujawsko-pomorskie'

### Equipment

In [26]:
data_json['props']['pageProps']['advert']['equipment']

[{'key': 'audio_and_computing',
  'label': 'Audio i multimedia',
  'values': [{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'},
   {'key': 'radio', 'label': 'Radio'},
   {'key': 'hands_free_system', 'label': 'Zestaw głośnomówiący'},
   {'key': 'usb_in', 'label': 'Gniazdo USB'},
   {'key': 'navigation_system', 'label': 'System nawigacji satelitarnej'}]},
 {'key': 'comfort_and_addons',
  'label': 'Komfort i dodatki',
  'values': [{'key': 'air_conditioning_type',
    'label': 'Klimatyzacja automatyczna'},
   {'key': 'steering_wheel_with_radio_operation',
    'label': 'Kierownica ze sterowaniem radia'},
   {'key': 'multi_functional_steering_wheel',
    'label': 'Kierownica wielofunkcyjna'},
   {'key': 'power_windows_front', 'label': 'Elektryczne szyby przednie'}]},
 {'key': 'electronics_and_driver_assistance',
  'label': 'Systemy wspomagania kierowcy',
  'values': [{'key': 'cruisecontrol_type', 'label': 'Tempomat'},
   {'key': 'door_mirror_electrically_adjustable_in_general',

In [27]:
equipment = data_json['props']['pageProps']['advert']['equipment']
equipment[0]

{'key': 'audio_and_computing',
 'label': 'Audio i multimedia',
 'values': [{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'},
  {'key': 'radio', 'label': 'Radio'},
  {'key': 'hands_free_system', 'label': 'Zestaw głośnomówiący'},
  {'key': 'usb_in', 'label': 'Gniazdo USB'},
  {'key': 'navigation_system', 'label': 'System nawigacji satelitarnej'}]}

In [28]:
equipment[1]

{'key': 'comfort_and_addons',
 'label': 'Komfort i dodatki',
 'values': [{'key': 'air_conditioning_type',
   'label': 'Klimatyzacja automatyczna'},
  {'key': 'steering_wheel_with_radio_operation',
   'label': 'Kierownica ze sterowaniem radia'},
  {'key': 'multi_functional_steering_wheel',
   'label': 'Kierownica wielofunkcyjna'},
  {'key': 'power_windows_front', 'label': 'Elektryczne szyby przednie'}]}

In [29]:
equipment[0]['values']

[{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'},
 {'key': 'radio', 'label': 'Radio'},
 {'key': 'hands_free_system', 'label': 'Zestaw głośnomówiący'},
 {'key': 'usb_in', 'label': 'Gniazdo USB'},
 {'key': 'navigation_system', 'label': 'System nawigacji satelitarnej'}]

In [30]:
equipment[0]['values'][0]

{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'}

In [31]:
[i['label'] for i in equipment[0]['values']]

['Interfejs Bluetooth',
 'Radio',
 'Zestaw głośnomówiący',
 'Gniazdo USB',
 'System nawigacji satelitarnej']

In [32]:
eq = []
for e in equipment:
    print(e)
    print(e['values'])
    equipment_by_category = [i['label'] for i in e['values']]
    print(equipment_by_category)
    for ebc in equipment_by_category:
        eq.append(ebc)
    print('----')

{'key': 'audio_and_computing', 'label': 'Audio i multimedia', 'values': [{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'}, {'key': 'radio', 'label': 'Radio'}, {'key': 'hands_free_system', 'label': 'Zestaw głośnomówiący'}, {'key': 'usb_in', 'label': 'Gniazdo USB'}, {'key': 'navigation_system', 'label': 'System nawigacji satelitarnej'}]}
[{'key': 'bluetooth_interface', 'label': 'Interfejs Bluetooth'}, {'key': 'radio', 'label': 'Radio'}, {'key': 'hands_free_system', 'label': 'Zestaw głośnomówiący'}, {'key': 'usb_in', 'label': 'Gniazdo USB'}, {'key': 'navigation_system', 'label': 'System nawigacji satelitarnej'}]
['Interfejs Bluetooth', 'Radio', 'Zestaw głośnomówiący', 'Gniazdo USB', 'System nawigacji satelitarnej']
----
{'key': 'comfort_and_addons', 'label': 'Komfort i dodatki', 'values': [{'key': 'air_conditioning_type', 'label': 'Klimatyzacja automatyczna'}, {'key': 'steering_wheel_with_radio_operation', 'label': 'Kierownica ze sterowaniem radia'}, {'key': 'multi_functiona

In [33]:
# collective equipment list
print(eq)

['Interfejs Bluetooth', 'Radio', 'Zestaw głośnomówiący', 'Gniazdo USB', 'System nawigacji satelitarnej', 'Klimatyzacja automatyczna', 'Kierownica ze sterowaniem radia', 'Kierownica wielofunkcyjna', 'Elektryczne szyby przednie', 'Tempomat', 'Lusterka boczne ustawiane elektrycznie', 'Podgrzewane lusterka boczne', 'Ogranicznik prędkości', 'Kontrola trakcji', 'Światła do jazdy dziennej diodowe LED', 'Wspomaganie kierownicy', 'Felgi stalowe', 'ABS', 'ESP', 'Elektroniczny system rozdziału siły hamowania', 'System wspomagania hamowania', 'Poduszka powietrzna kierowcy', 'Poduszka powietrzna pasażera', 'Boczna poduszka powietrzna kierowcy', 'Boczne poduszki powietrzne - przód', 'Isofix (punkty mocowania fotelika dziecięcego)']


### Aggregate information

In [34]:
offer_data = {}

adv = data_json['props']['pageProps']['advert']

price = float(adv['price']['value'])
description = adv['description'].replace('\n', ' ')
seller_province = adv['seller']['location']['region']
offer_params = adv['details']

for i in offer_params:
    label = i['label']
    value = i['value']
    offer_data[label] = value

offer_data['cena'] = price
offer_data['opis'] = description
offer_data['Wojewodztwo_sprzedajacego'] = seller_province

In [35]:
offer_data

{'Oferta od': 'Prywatne',
 'Importowany': 'Tak',
 'Pokaż oferty z numerem VIN': 'Tak',
 'Ma numer rejestracyjny': 'Tak',
 'Marka pojazdu': 'Citroën',
 'Model pojazdu': 'C3',
 'Wersja': '1.4 HDi Attraction',
 'Generacja': 'II (2008-2018)',
 'Rok produkcji': '2014',
 'Przebieg': '167 500 km',
 'Pojemność skokowa': '1 398 cm3',
 'VIN': 'VF7SC8HR4EW648604',
 'Rodzaj paliwa': 'Diesel',
 'Moc': '68 KM',
 'Skrzynia biegów': 'Manualna',
 'Napęd': 'Na przednie koła',
 'Spalanie Poza Miastem': '3.0 l/100km',
 'Spalanie W Mieście': '4 l/100km',
 'Typ nadwozia': 'Auta małe',
 'Emisja CO2': '99 g/km',
 'Liczba drzwi': '5',
 'Liczba miejsc': '2',
 'Kolor': 'Srebrny',
 'Rodzaj koloru': 'Metalik',
 'Kraj pochodzenia': 'Francja',
 'Data pierwszej rejestracji w historii pojazdu': '24 wrzesień 2014',
 'Zarejestrowany w Polsce': 'Tak',
 'Stan': 'Używane',
 'cena': 12999.0,
 'opis': '!!! WERSJA 2 OSOBOWA - BRAK TYLNEJ KANAPY !!!  CITROEN C3 II HATCHBACK SOCIETE PO FACELIFTINGU 1.4 HDi 68 KM (50kW) DIESEL  

## Functions

In [36]:
def get_offers(page_number):
    '''
    This function returns a part of HTML text that contains a link to the advertisements.
    The URL for each car type has been pasted to avoid errors.
    '''
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-mini/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-city-car/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-coupe/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-cabrio/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-combi/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-compact/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-minivan/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    #url = f"https://www.otomoto.pl/osobowe/uzywane/seg-sedan/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"
    url = f"https://www.otomoto.pl/osobowe/uzywane/seg-suv/od-2000?search%5Bfilter_enum_damaged%5D=0&search%5Bfilter_enum_registered%5D=1&search%5Bfilter_float_mileage%3Ato%5D=500000&page={page_number}&search%5Badvanced_search_expanded%5D=true"

    response = requests.get(url)
    sleep(0.5)
    content = response.text
    soup = BeautifulSoup(content, 'html.parser')
    offers = soup.find_all('h1', {'class': "ev7e6t89 ooa-1xvnx1e er34gjf0"})
    return offers


def get_offers_url(offers):
    '''
    This function returns link to the advertisements.
    '''
    offers_url = []
    for offer in offers:
        offer_url = offer.find('a')['href']
        if offer_url.startswith('https://www.otomoto.pl'):
            offers_url.append(offer_url)
    return offers_url


def get_offer(offer_url):
    '''
    This function returns JSON with advertisement information.
    '''
    offer_response = requests.get(url=offer_url)
    sleep(0.5)
    offer_content = offer_response.text
    offer_soup = BeautifulSoup(offer_content, 'html.parser')

    try:
        text = offer_soup.find('script', {'id': "__NEXT_DATA__"}).get_text()
        data_json = json.loads(text)
        return data_json
    except:
        pass



def get_offer_data(offer_json):
    '''
    This function returns dictionary with selected advertisement information.
    '''
    adv = offer_json['props']['pageProps']['advert']
    offer_data = {}

    price = float(adv['price']['value'])
    description = adv['description'].replace('\n', ' ')
    seller_province = adv['seller']['location']['region']
    offer_params = adv['details']

    for i in offer_params:
        label = i['label']
        value = i['value']
        offer_data[label] = value

    offer_data['cena'] = price
    offer_data['opis'] = description
    offer_data['Wojewodztwo_sprzedajacego'] = seller_province

    return offer_data

## Scraping

In [37]:
data = []

In [38]:
for page_number in range(1, 5):
    offers = get_offers(page_number)
    offers_url = get_offers_url(offers)
    for offer_url in offers_url:
        try:
            offer_json = get_offer(offer_url)
            offer_data = get_offer_data(offer_json)
            data.append(offer_data)
        except:
            pass

    print(f"Page number: {page_number}")
    print(f"Number of offers: {len(data)}")
    print(40 * '-')


df = pd.DataFrame(data)
df.drop_duplicates(inplace=True)
car_type = data[0]['Typ nadwozia']
df.to_csv(f'otomoto_{len(data)}_{car_type}.csv')

Page number: 1
Number of offers: 32
----------------------------------------
Page number: 2
Number of offers: 64
----------------------------------------
Page number: 3
Number of offers: 96
----------------------------------------
Page number: 4
Number of offers: 128
----------------------------------------


In [39]:
len(data)

128

In [40]:
data[0]

{'Oferta od': 'Prywatne',
 'Importowany': 'Tak',
 'Pokaż oferty z numerem VIN': 'Tak',
 'Marka pojazdu': 'Audi',
 'Model pojazdu': 'Q3',
 'Wersja': '45 TFSI Quattro S Line S tronic',
 'Rok produkcji': '2021',
 'Przebieg': '27 000 km',
 'Pojemność skokowa': '1 984 cm3',
 'VIN': 'WA1FECF32M1031800',
 'Rodzaj paliwa': 'Benzyna',
 'Moc': '230 KM',
 'Skrzynia biegów': 'Automatyczna',
 'Napęd': '4x4 (dołączany automatycznie)',
 'Spalanie W Mieście': '9 l/100km',
 'Typ nadwozia': 'SUV',
 'Liczba drzwi': '5',
 'Liczba miejsc': '5',
 'Kolor': 'Czarny',
 'Rodzaj koloru': 'Metalik',
 'Leasing': 'Tak',
 'Zarejestrowany w Polsce': 'Tak',
 'Bezwypadkowy': 'Tak',
 'Serwisowany w ASO': 'Tak',
 'Stan': 'Używane',
 'cena': 169900.0,
 'opis': 'Na sprzedaż piękne Audi Q3 z 2021 roku z automatyczną 8 stopniową skrzynią biegów z napędem Quattro.  Auto w bardzo bogatej wersji wyposażeniowej.  Użytkowane prywatnie. Nie wymaga żadnego wkladu finansowego. W perfekcyjnym stanie zarówno wizaulanym jak i techniczn

In [41]:
df.duplicated().sum()

0

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125 entries, 0 to 127
Data columns (total 46 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Oferta od                                      125 non-null    object 
 1   Importowany                                    39 non-null     object 
 2   Pokaż oferty z numerem VIN                     124 non-null    object 
 3   Marka pojazdu                                  125 non-null    object 
 4   Model pojazdu                                  125 non-null    object 
 5   Wersja                                         97 non-null     object 
 6   Rok produkcji                                  125 non-null    object 
 7   Przebieg                                       125 non-null    object 
 8   Pojemność skokowa                              123 non-null    object 
 9   VIN                                            124 non